<a href="https://colab.research.google.com/github/shrigoa/random/blob/main/Quvia_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""Random.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/github/shrigoa/random/blob/main/Random.ipynb

Problem:
1. Manage internet bandwidth usage on a ship.

Important data:
1. Access to two communication links each contracted with 15TB of bandwidth for a month, thus, total of 30TB.

Goals:
1. Optimal usage: Bandwidth should be consumed only when required and minimized when not needed.
2.a. Fairness Across Time: Quality of Experience (QoE) should be consistent across different timeslots.
2.b. Significant variations — such as excellent QoE in some periods and poor QoE in others — must be inimized to avoid service quality alarms.
2.c. Demand met percentage is the QoE. It should not vary much.

Given Data:
1. Per hour demand data for upstream and downstream traffic
- Variations of 20-25% are possible
- Columns: Time slots, downstream demand, upstream demand
2. Ship arrival at ports
- Variation of 1-2 hours earlier or later.
- Demand drops 50% at port compared to sea.
- Columns: Port, Arrival Time, Departure Time
3. Link Downtime
- One of the links will be unavailable on certain days.
- This downtime must be configurable in the code.
- During downtime, the algorithm must reserve enough bandwidth on the remaining active link to maintain stable QoE and avoid service degradation.
4. Consumption Limits:
- Each link supports maximum bandwidth of 500Mbps at any given point.
- Bandwidth planning is done in 5 min time-slots.
- Bandwidth volume (e.g., 1 GB in 5 minutes) will internally be converted into rate for enforcement.
5. End of the month requirement:
- Allocated 30TB should be completely used. (CONSTRAINT)
- No under/over usage allowed.
6. Objective:
- Demand met percentage is the QoE (OBJECTIVE FUNCTION).
- Needs to be as high as possible.

7. Parameters:
- Number of 5 mins slots per day: 288 slots
- Number of slots per hour: 12 slots
- Number of days: 30 days
- Seconds in each slot:
- Downtime start day: 19
- Downtime end day: 27
- Capacity per link: 15 TB
- Number of links: 2
- Number of links up during downtime: 1
- Maximum bandwidth of any link: 500 Mbps

7. Decision Variables
- How much data is consumed in each 5 mins slot: data_consumed[day, hour, slot]

8. Objective Function
- demand[day, hour] - sum of data_consumed[day, hour, slot] over all slots for each day, for each hour data
"""
#Install packages
!pip install gurobipy

#Import packages
import random
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import gurobipy as gp
from gurobipy import GRB
from datetime import datetime, timedelta

#---------------------------------------------------------------------------------------------------------------------------------

#Parameters
#Fixed Parameters
hours_in_day = 24
mins_in_hour = 60
secs_in_min = 60

#Configurable Parameters
slot_mins = 5
planning_start_date = datetime(2025, 5, 1)
num_planning_days = 30
active_links = 2
cap_per_link = 15000
max_bandwidth_per_link = 0.5
downtime_start_day = datetime(2025, 5, 20)
downtime_end_day = datetime(2025, 5, 28)
active_links_downtime = 1
allowed_QoE_variance_percent = 0.01
days_to_consider = 3

#Derived Parameters
secs_in_hour = mins_in_hour * secs_in_min
planning_end_date = planning_start_date + timedelta(days = num_planning_days - 1)
seconds_in_slot = slot_mins * secs_in_min
seconds_in_hour = mins_in_hour * secs_in_min
slots_per_hour = int(mins_in_hour/slot_mins)
slots_per_day = int(slots_per_hour * hours_in_day)

total_cap = cap_per_link * active_links
total_cap_downtime = cap_per_link * active_links_downtime

max_data_per_slot_per_link = max_bandwidth_per_link * seconds_in_slot
max_data_per_hour_per_link = max_data_per_slot_per_link * slots_per_hour

max_data_per_slot = max_data_per_slot_per_link * active_links
max_data_per_hour = max_data_per_hour_per_link * active_links

downtime_max_data_per_slot = max_data_per_slot_per_link * active_links_downtime
downtime_max_data_per_hour = max_data_per_hour_per_link * active_links_downtime

downtime_start_day_num = (downtime_start_day - planning_start_date).days
downtime_end_day_num = (downtime_end_day - planning_start_date).days

max_bandwidth = max_bandwidth_per_link * active_links
downtime_max_bandwidth = max_bandwidth_per_link * active_links_downtime

allowed_QoE_variance = allowed_QoE_variance_percent/100
#---------------------------------------------------------------------------------------------------------------------------------

#Data Creation
try:
  input_data_df = pd.read_csv("Input Demand.csv")
except:
  input_data_df = pd.DataFrame(columns = ['Day', 'Date', 'Hour', 'Downstream Demand', 'Upstream Demand', 'Total Demand', 'Variation %', 'Final Demand', 'Demand Met', 'QoE %', 'Proposed Consumption Per Slot', 'Proposed Slot Speed'])
  for day in range(num_planning_days):
    date = planning_start_date + timedelta(days=day)
    for hour in range(hours_in_day):
      downstream_demand = random.randint(10, 40)
      upward_demand = random.randint(10, 40)
      total_demand = downstream_demand + upward_demand
      variation_percentage = 20 if 9 <= hour < 21 else 10
      final_demand = round((1 + variation_percentage/100) * total_demand)
      input_data_df.loc[len(input_data_df)] = [day, date, hour, downstream_demand, upward_demand, total_demand, variation_percentage, final_demand, "", "", "", ""]
  input_data_df.to_csv("Input Demand.csv", index=False)

#Replacing 0 demand with 1 GB Demand
input_data_df['Final Demand'] = input_data_df['Final Demand'].mask(input_data_df['Final Demand']==0).fillna(1)

#Total percentage of demand that would be fulfilled
final_demand =  input_data_df.loc[:,'Final Demand'].sum()
total_fulfilled_percentage = total_cap / final_demand if total_cap / final_demand < 1 else 1
#---------------------------------------------------------------------------------------------------------------------------------

#Define Model
env = gp.Env()
#env.setParam('OutputFlag', 0)
mip = gp.Model('PlanningMIP', env = env)
#-----------------------------------------------------------------------------------------------------------------------------------

#Decision Variables
hour_consumption = {}
QoE = {}

for day in range(num_planning_days):
  for hour in range(hours_in_day):
    max_available_data_per_hour = downtime_max_data_per_hour if downtime_start_day_num <= day <= downtime_end_day_num else max_data_per_hour
    max_available_data_per_slot = downtime_max_data_per_slot if downtime_start_day_num <= day <= downtime_end_day_num else max_data_per_slot

    hour_consumption[day, hour] = mip.addVar(lb = 0, ub = max_available_data_per_hour, vtype=GRB.CONTINUOUS, name = f'hour_consumption_{day}_{hour}')
    QoE[day, hour] = mip.addVar(lb = 0, ub = max_available_data_per_hour, vtype=GRB.CONTINUOUS, name = f'qoe_{day}_{hour}')

sum_demand_met =  gp.quicksum(hour_consumption[day, hour] for hour in range(hours_in_day) for day in range(num_planning_days))
average_demand_met = sum_demand_met/(num_planning_days * hours_in_day)
#-----------------------------------------------------------------------------------------------------------------------------------

#Constraints
#QoE and Hour Consumption Relation
for day in range(num_planning_days):
  date = planning_start_date + timedelta(days=day)
  for hour in range(hours_in_day):
    max_bandwidth = max_bandwidth_per_link
    max_available_data_per_slot = downtime_max_data_per_slot if downtime_start_day_num <= day <= downtime_end_day_num else max_data_per_slot

    current_hour_demand = input_data_df[(input_data_df['Day'] == day) & (input_data_df['Hour'] == hour)]['Final Demand']
    mip.addConstr(QoE[day, hour] <= hour_consumption[day, hour]/current_hour_demand, name=f'QoE_Hour_Consumption')

#Slot and Hour Consumption
for day in range(num_planning_days):
  for hour in range(hours_in_day):
    allowed_bandwidth = downtime_max_bandwidth if downtime_start_day_num <= day <= downtime_end_day_num else max_bandwidth
    mip.addConstr(hour_consumption[day, hour]/seconds_in_hour <= allowed_bandwidth, name=f'bandwidth_limits')

#Total Capacity Constraint
data_consumed_exp = gp.LinExpr()
for day in range(num_planning_days):
  for hour in range(hours_in_day):
    data_consumed_exp += hour_consumption[day, hour]
mip.addConstr(data_consumed_exp == total_cap, name=f'total_data_consumed')

#Maximum Capacity Allowed during Downtime Constraint
downtime_data_consumed_exp = gp.LinExpr()
for day in range(downtime_start_day_num, downtime_end_day_num + 1):
  for hour in range(hours_in_day):
    downtime_data_consumed_exp += hour_consumption[day, hour]
mip.addConstr(downtime_data_consumed_exp <= total_cap_downtime, name=f'downtime_data_consumed')

#Variance Constraint
previous_hour_demand = input_data_df[(input_data_df['Day'] == num_planning_days - 1)  & (input_data_df['Hour'] == hours_in_day - 1)]['Final Demand']
previous_hour_consumption = hour_consumption[num_planning_days - 1, hours_in_day - 1]

previous_hour_qoe = QoE[num_planning_days - 1, hours_in_day - 1]
for day in range(num_planning_days):
  for hour in range(hours_in_day):
    mip.addLConstr(QoE[day, hour], GRB.GREATER_EQUAL, previous_hour_qoe - allowed_QoE_variance, name=f'unmet_demand_constraint')
    mip.addLConstr(QoE[day, hour], GRB.LESS_EQUAL, previous_hour_qoe + allowed_QoE_variance, name=f'unmet_demand_constraint')
    previous_hour_qoe = QoE[day, hour]
#--------------------------------------------------------------------------------------------------------------------------------------------------

#Objective
qoe_exp = gp.LinExpr()
for day in range(num_planning_days):
  for hour in range(hours_in_day):
    qoe_exp += QoE[day, hour]

#----------------------------------------------------------------------------------------------------------------------------------------------
#Solve Model
mip.setObjective(qoe_exp, GRB.MAXIMIZE)
mip.write('mip.lp')
mip.optimize()

#----------------------------------------------------------------------------------------------------------------------------------------------
#Result
if mip.status == GRB.OPTIMAL:
  print("Found Optimal Solution!!")
  print("Objective", mip.ObjVal)

  counter = 0
  for day in range(num_planning_days):
    for hour in range(hours_in_day):
      input_data_df.loc[counter, "Demand Met"] = hour_consumption[day, hour].X
      input_data_df.loc[counter, "QoE %"] = QoE[day, hour].X * 100
      input_data_df.loc[counter, "Proposed Consumption Per Slot"] = hour_consumption[day, hour].X/slots_per_hour
      input_data_df.loc[counter, "Proposed Slot Speed"] = (input_data_df.loc[counter, "Proposed Consumption Per Slot"] * 1000)/seconds_in_slot
      counter += 1
elif mip.status == GRB.INFEASIBLE:
  print('Infeasible Model')
else:
  print('Sub-Optimal')
#----------------------------------------------------------------------------------------------------------------------------------------------

#Write Output
input_data_df.to_csv("Demand and Output.csv", index = False)
#----------------------------------------------------------------------------------------------------------------------------------------------


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 95.1 MB/s eta 0:00:00
Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads



GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information

In [ ]:

#---------------------------------------------------------------------------------------------------------------------------------

#Data Creation
monthly_planning_df = pd.read_csv("Monthly Planning Output.csv")
print()

input_data_df = pd.DataFrame(columns=['Day', 'Date', 'Hour', 'Slot', 'Total Demand', 'Variation %', 'Final Demand', 'Demand Met', 'QoE %', 'Proposed Consumption Per Slot', 'Proposed Slot Speed', 'Actual Demand'])
counter = 0
for day in range(num_planning_days):
  date = planning_start_date + timedelta(days=day)
  for hour in range(hours_in_day):
    row = monthly_planning_df[(monthly_planning_df['Day'] == day) & (monthly_planning_df['Hour'] == hour)]
    total_demand = row['Total Demand'].iloc[0]
    variation_percentage = row['Variation %'].iloc[0]
    final_demand = row['Final Demand'].iloc[0]
    demand_met = row['Demand Met'].iloc[0]
    qoe_percentage = row['QoE %'].iloc[0]
    proposed_consumption_per_slot = row['Proposed Consumption Per Slot'].iloc[0]
    proposed_slot_speed = row['Proposed Slot Speed'].iloc[0]

    for slot in range(slots_per_hour):
      input_data_df.loc[len(input_data_df)] = [day, date, hour, slot, total_demand, variation_percentage, final_demand, demand_met, qoe_percentage, proposed_consumption_per_slot, proposed_slot_speed, ""]

  if counter >= days_to_consider:
    break
  counter += 1


#Write Output
input_data_df.to_csv("Live Execution Output.csv", index = False)